In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Bernoulli, Normal, Categorical, Dirichlet, Uniform, Multinomial, Empirical
from edward.util import Progbar

import matplotlib.pyplot as plt
%matplotlib inline

Create some sample data:

In [2]:
sample_size = 1000
obs_cond = np.random.choice([0,1,2],[sample_size],p=[.7,.1,.2]) # cold = 0, flu = 1, allergies = 2

runny_nose_cold = np.random.choice([0,1],[sample_size],p=[.1,.9])
headache_cold = np.random.choice([0,1],[sample_size],p=[.5,.5])
fever_cold = np.random.choice([0,1],[sample_size],p=[.99,.01])

runny_nose_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])
headache_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])
fever_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])

runny_nose_al = np.random.choice([0,1],[sample_size],p=[.1,.9])
headache_al = np.random.choice([0,1],[sample_size],p=[.99,.01])
fever_al = np.random.choice([0,1],[sample_size],p=[.99,.01])

runny_nose = np.stack([runny_nose_cold,runny_nose_flu,runny_nose_al])
headache = np.stack([headache_cold,headache_flu,headache_al])
fever = np.stack([fever_cold,fever_flu,fever_al])

In [3]:
obs_runny_nose = np.array([runny_nose[j,i] for i,j in enumerate(obs_cond)])
obs_headache = np.array([headache[j,i] for i,j in enumerate(obs_cond)])
obs_fever = np.array([fever[j,i] for i,j in enumerate(obs_cond)])

Hide some of the sample data:

In [4]:
msk_cond = np.ma.masked_where(np.random.rand(sample_size) > .9, obs_cond)
msk_runny_nose = np.ma.masked_where(np.random.rand(sample_size) > .9, obs_runny_nose)
msk_headache = np.ma.masked_where(np.random.rand(sample_size) > .9, obs_headache)
msk_fever = np.ma.masked_where(np.random.rand(sample_size) > .9, obs_fever)

In [5]:
msk_cond.set_fill_value(3)
msk_runny_nose.set_fill_value(2)
msk_headache.set_fill_value(2)
msk_fever.set_fill_value(2)

In [6]:
a = np.concatenate([np.eye(3,dtype=np.float32),np.zeros([1,3],dtype=np.float32)])
b = np.concatenate([np.zeros([3,3],dtype=np.float32),np.ones([1,3],dtype=np.float32)])

In [7]:
#cat_p = Dirichlet(alpha=tf.constant([1.,1.,1.])) #tf.ones([sample_size,3])*
#cond = Categorical(name='cond', p=tf.ones([sample_size,3])*cat_p) 
cond = Categorical(name='cond', p=tf.ones([sample_size,3])*tf.constant([.7,.1,.2])) 
    
W = tf.Variable(tf.zeros([3,3]),name='W') #Uniform(name='W',a=0.,b=1.,shape=[3,3])

#W = Normal(mu=tf.zeros([3,3]),name='W',sigma=tf.ones([3,3])) #Uniform(name='W',a=0.,b=1.,shape=[3,3])
    
runny_nose = Bernoulli(name='runny_nose',logits=tf.gather(W[0,:],cond))
headache = Bernoulli(name='headache',logits=tf.gather(W[1,:],cond))
fever = Bernoulli(name='fever',logits=tf.gather(W[2,:],cond))

In [8]:
q
inference = ed.MAP({},
                   {cond_out:msk_cond.filled(),runny_nose:obs_runny_nose,headache:obs_headache,fever:obs_fever})

In [9]:
inference.run(n_iter=1000)

1000/1000 [100%] ██████████████████████████████ Elapsed: 2s | Loss: inf


In [10]:
tf.nn.sigmoid(W).eval()

array([[ 0.86261046,  0.87524354,  0.86035675],
       [ 0.40516001,  0.4085269 ,  0.41162029],
       [ 0.06396348,  0.05525392,  0.06415366]], dtype=float32)

In [11]:
plt.hist(tf.sigmoid(hide_prob.sample(100)).eval())

NameError: name 'hide_prob' is not defined

In [ ]:
cond.eval()

In [ ]:

plt.hist([cond.eval()[1] for i in range(100)])

In [ ]:
tf.gather((tf.constant(a.T)*(1-.1)+tf.constant(b.T)*.1),[0,2,1,1,1,0]).eval()